# Parse Txt

In [1]:
import PyPDF2 as pydf
import pandas as pd
from pandas import DataFrame as DF, Series
import numpy as np
import re

In [2]:
path = 'results-5-2010.pdf'
pdf = open(path, 'rb')

In [3]:
reader = pydf.PdfFileReader(pdf)

In [4]:
page = reader.getPage(2)

In [5]:
print(page.extractText())

May 18, 2010, Primary Election Abstracts of Votes * Nominee 
** Elected WI = Write In
 3REPRESENTATIVE IN CONGRESS, 1ST DISTRICT Democrat  Robinson *Wu Misc. 
County David David  
 Clatsop 888 3,937 25 

Columbia 1,510 4,495 46 
Multnomah 1,812 11,843 31 
Washington 8,029 34,593 201 
Yamhill 1,863 6,571 80 
 TOTAL 14,102 61,439 383 
REPRESENTATIVE IN CONGRESS, 1ST DISTRICT Republican  Keller Kuzmanich Brodhead *Cornilles Misc. 
County Douglas Fitzgerald John Stephan Andrew Rob  
 Clatsop 732 681 94 1,146 37 
Columbia 1,110 1,146 121 1,091 23 
Multnomah 776 971 50 1,388 26 
Washington 9,886 9,070 725 13,764 163 
Yamhill 2,281 2,596 223 4,052 50 
 TOTAL 14,785 14,464 1,213 21,441 299 



In [6]:
pages = []
for i in range(reader.numPages):
    pages.append(reader.getPage(i).extractText())

In [7]:
pages[1]

'May 18, 2010, Primary Election Abstracts of Votes * Nominee \n** Elected WI = Write In\n 2UNITED STATES SENATOR Republican  Stutzman Parker Later Dinkel \n*Huffman Woodland Waldron Misc. \nCounty Tom Robin S Loren G Shane Jim Walter H       Keith  \nBaker 235 106 375 364 587 32 245 18 \nBenton 797 268 919 702 2,207 86 431 51 \nClackamas 2,940 1,281 3,590\n 3,054 13,624 382 2,582 162 \nClatsop 345 199 302 276 1,044 45 259 45 \nColumbia 383 171 485 568 1,555 66 238 43 \nCoos 843 466 979 830 1,773 128 598 56 \n\nCrook 255 122 329 441 894 60 332 28 \nCurry 339 188 516 539 746 55 258 33 \nDeschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 \nDouglas 1,362 652 1,983 1,901 4,784 208 1,314 113 \nGilliam 20 12 22 28 171 0 11 1 \n\nGrant 90 41 93 209 590 18 81 14 \nHarney 147 71 184 212 280 15 153 19 \nHood River 201 104 148 164 574 21 122 11 \nJackson 1,429 925 3,549 2,838 5,250 251 1,603 106 \nJefferson 116 67 144 297 1,168 34 121  9 \n\nJosephine 890 570 1,568 1,661 2,535 192 1,075 102 \nKla

In [8]:
def get_table_starts(p):
    """ Return list of string indexes where tables begin
        on given page (p).
    """
    return [m.span()[1] for m in re.finditer(r'Democrat|Republican', pages[p])]

In [9]:
def get_table_strings(p, idxs):
    n = len(idxs)
    return [pages[p][idxs[i]:idxs[i+1]-1].lstrip() if i < n-1
            else pages[p][idxs[i]:].lstrip() for i in range(n)]  

In [ ]:
def get_table_rows(p, strings):
    tables = []
    for s in strings:
        

In [10]:
idxs = get_table_starts(1)
strings = get_table_strings(1, idxs)

In [12]:
lines = [l.strip() for l in re.sub('\n\s*(?=\d)', ' ', strings[0]).split('\n') if l != '']
lines = [re.sub('\s{2,}', ' ', l) for l in lines]
last_col_line = [i for i,l in enumerate(lines) if l.startswith('County')][0]
last_row_line = [i for i,l in enumerate(lines) if l.startswith('TOTAL')][0]
rows = [re.sub('(?!\d), (?=\d)', '', r) for r in lines[last_col_line+1:last_row_line+1]]
rows = [re.sub('\s+(?=\D)', '-', l).replace(',', '').split(' ') for l in rows]
# convert numeric elements to int
rows = [[int(el) if el.isdigit() else el for el in r] for r in rows]
columns = [l.strip('County ') for l in lines[:last_col_line+1]]

In [13]:
DF(rows)

,0,1,2,3,4,5,6,7,8
0,Baker,235,106,375,364,587,32,245,18
1,Benton,797,268,919,702,2207,86,431,51
2,Clackamas,2940,1281,3590,3054,13624,382,2582,162
3,Clatsop,345,199,302,276,1044,45,259,45
4,Columbia,383,171,485,568,1555,66,238,43
5,Coos,843,466,979,830,1773,128,598,56
6,Crook,255,122,329,441,894,60,332,28
7,Curry,339,188,516,539,746,55,258,33
8,Deschutes,1053,813,2161,2103,6677,189,1264,107
9,Douglas,1362,652,1983,1901,4784,208,1314,113


In [14]:
columns

['Stutzman Parker Later Dinkel',
 '*Huffman Woodland Waldron Misc.',
 'Tom Robin S Loren G Shane Jim Walter H Keith']

In [15]:
print(strings[0])

Stutzman Parker Later Dinkel 
*Huffman Woodland Waldron Misc. 
County Tom Robin S Loren G Shane Jim Walter H       Keith  
Baker 235 106 375 364 587 32 245 18 
Benton 797 268 919 702 2,207 86 431 51 
Clackamas 2,940 1,281 3,590
 3,054 13,624 382 2,582 162 
Clatsop 345 199 302 276 1,044 45 259 45 
Columbia 383 171 485 568 1,555 66 238 43 
Coos 843 466 979 830 1,773 128 598 56 

Crook 255 122 329 441 894 60 332 28 
Curry 339 188 516 539 746 55 258 33 
Deschutes 1,053 813 2,161 2,103 6,677 189 1,264 107 
Douglas 1,362 652 1,983 1,901 4,784 208 1,314 113 
Gilliam 20 12 22 28 171 0 11 1 

Grant 90 41 93 209 590 18 81 14 
Harney 147 71 184 212 280 15 153 19 
Hood River 201 104 148 164 574 21 122 11 
Jackson 1,429 925 3,549 2,838 5,250 251 1,603 106 
Jefferson 116 67 144 297 1,168 34 121  9 

Josephine 890 570 1,568 1,661 2,535 192 1,075 102 
Klamath 640 501 1,419 1,236 1,978 145 1,005 53 
Lake 195 90 215 203 241 19 187  8 
Lane 2,590 1,385 3,358 3,372 9,528 517 1,396 241 
Lincoln 554 241 526